In [22]:
from mxnet import autograd
from mxnet import gluon
from mxnet import image
from mxnet import init
from mxnet import nd
from mxnet.gluon.data import vision
import numpy as np
import random
import mxnet as mx
from mxnet.gluon import nn
import pandas as pd
ctx = mx.gpu(0)

In [23]:
train = pd.read_json("train.json")
test = pd.read_json("test.json")

In [24]:
test.head()

,band_1,band_2,id,inc_angle
0,"[-15.863251, -15.201077, -17.887735, -19.17248...","[-21.629612, -21.142353, -23.908337, -28.34524...",5941774d,34.966400
1,"[-26.058969497680664, -26.058969497680664, -26...","[-25.754207611083984, -25.754207611083984, -25...",4023181e,32.615072
2,"[-14.14109992980957, -15.064241409301758, -17....","[-14.74563980102539, -14.590410232543945, -14....",b20200e4,37.505433
3,"[-12.167478, -13.706167, -16.54837, -13.572674...","[-24.32222, -26.375538, -24.096739, -23.8769, ...",e7f018bb,34.473900
4,"[-23.37459373474121, -26.02718162536621, -28.1...","[-25.72234344482422, -27.011577606201172, -23....",4371c8c3,43.918874


In [25]:
trainy = train['is_iceberg']

In [26]:
X_band_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_1"]])
X_band_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_2"]])
trainx = np.concatenate([X_band_1[:, :72, :72, np.newaxis], X_band_2[:, :72, :72, np.newaxis]], axis=-1)

x_band_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_1"]])
x_band_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_2"]])
testx = np.concatenate([x_band_1[:, :72, :72, np.newaxis], x_band_2[:, :72, :72, np.newaxis]], axis=-1)

In [27]:
print (trainx.shape,testx.shape)

(1604, 72, 72, 2) (8424, 72, 72, 2)


In [28]:
trainx = trainx.transpose(0,3,1,2)
testx = testx.transpose(0,3,1,2)

In [29]:
meant = trainx.mean()
vart = trainx.var()
print (meant,vart)
trainx = (trainx - meant) / vart
testx = (testx - meant) / vart
meant = trainx.mean()
vart = trainx.var()
print (meant,vart)

-23.4544 27.7081
2.55198e-07 0.0360906


In [30]:
from sklearn.model_selection import train_test_split
xtrain, xval, ytrain, yval = train_test_split(trainx, trainy, test_size=0.1, random_state=42)
print (xtrain.shape,xval.shape,ytrain.shape,yval.shape)

(1443, 2, 72, 72) (161, 2, 72, 72) (1443,) (161,)


In [31]:
xtrain,ytrain,xval,yval = mx.nd.array(xtrain),mx.nd.array(ytrain),mx.nd.array(xval),mx.nd.array(yval)

In [32]:
batch_size = 128
train_iter = mx.io.NDArrayIter(xtrain, ytrain, batch_size, shuffle=True)
val_iter = mx.io.NDArrayIter(xval, yval, batch_size)

In [46]:
"""
train
"""
import datetime
import utils
import sys


def in_list(e, l):
    for i in l:
        if i == e:
            return True
    else:
        return False


In [45]:
batch_size = 128
net = ResNet(2)
loss_f = gluon.loss.SoftmaxCrossEntropyLoss()

num_epochs = 20
learning_rate = 0.1
weight_decay = 1e-4
lr_period = [8, 16]
lr_decay=0.1
log_file = None

net.collect_params().initialize(mx.init.Xavier(), ctx=ctx, force_reinit=True)
net.hybridize()
w_key = []
trainer = gluon.Trainer(net.collect_params(), 'sgd', 
                        {'learning_rate': learning_rate, 'momentum': 0.9, 'wd': weight_decay})
i = 0
for epoch in range(num_epochs):
    train_loss = 0.
    train_acc = 0.
    if in_list(epoch, lr_period):
        trainer.set_learning_rate(trainer.learning_rate * lr_decay)
    for data, label in train_data:
        label = label.as_in_context(ctx)
        with autograd.record():
            output = net(data.as_in_context(ctx))
            loss = loss_f(output, label)
        loss.backward()
        trainer.step(data.shape[0])

        _loss = nd.mean(loss).asscalar()
        _acc = utils.accuracy(output, label)
        train_loss += _loss
        train_acc += _acc


    train_loss /= len(train_data)
    train_acc /= len(train_data)

    if valid_data is not None:
        valid_acc = utils.evaluate_accuracy(valid_data, net, ctx)
        epoch_str = ("epoch %d, loss %.5f, train_acc %.4f, valid_acc %.4f" 
                     % (epoch, train_loss, train_acc, valid_acc))
    else:
        epoch_str = ("epoch %d, loss %.5f, train_acc %.4f"
                    % (epoch, train_loss, train_acc))

In [13]:
data = mx.sym.var('data')
# first conv layer
conv1 = mx.sym.Convolution(data=data, kernel=(3,3), num_filter=20)
tanh1 = mx.sym.Activation(data=conv1, act_type="relu")
pool1 = mx.sym.Pooling(data=tanh1, pool_type="max", kernel=(2,2), stride=(2,2))
# second conv layer
conv2 = mx.sym.Convolution(data=pool1, kernel=(5,5), num_filter=50)
tanh2 = mx.sym.Activation(data=conv2, act_type="relu")
pool2 = mx.sym.Pooling(data=tanh2, pool_type="max", kernel=(2,2), stride=(2,2))
# first fullc layer
flatten = mx.sym.flatten(data=pool2)
fc1 = mx.symbol.FullyConnected(data=flatten, num_hidden=500)
tanh3 = mx.sym.Activation(data=fc1, act_type="relu")
# second fullc
fc2 = mx.sym.FullyConnected(data=tanh3, num_hidden=10)
# softmax loss
lenet = mx.sym.SoftmaxOutput(data=fc2, name='softmax')

In [43]:
# create a trainable module on GPU 0
resnet = mx.mod.Module(symbol=net, context=mx.gpu(0))
# train with the same
resnet.fit(train_iter,
                eval_data=val_iter,
                optimizer='sgd',
                optimizer_params={'learning_rate':0.1},
                eval_metric='acc',
                batch_end_callback = mx.callback.Speedometer(batch_size, 100),
                num_epoch=10)

AttributeError: 'ResNet' object has no attribute 'list_arguments'

In [57]:
test_iter = mx.io.NDArrayIter(mnist['test_data'], None, batch_size)
prob = lenet_model.predict(test_iter)
test_iter = mx.io.NDArrayIter(mnist['test_data'], mnist['test_label'], batch_size)
# predict accuracy for lenet
acc = mx.metric.Accuracy()
lenet_model.score(test_iter, acc)
print(acc)
assert acc.get()[1] > 0.98

NameError: name 'mnist' is not defined

In [37]:
class Residual(nn.HybridBlock):
    def __init__(self, channels, same_shape=True, **kwargs):
        super(Residual, self).__init__(**kwargs)
        self.same_shape = same_shape
        with self.name_scope():
            strides = 1 if same_shape else 2
            self.conv1 = nn.Conv2D(channels, kernel_size=3, padding=1, strides=strides) # w,h or w/2,h/2
            self.bn1 = nn.BatchNorm()
            self.conv2 = nn.Conv2D(channels, kernel_size=3, padding=1)  # not change w,h
            self.bn2 = nn.BatchNorm()
            if not same_shape:
                self.conv3 = nn.Conv2D(channels, kernel_size=1, strides=strides)
                
    def hybrid_forward(self, F, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        if not self.same_shape:
            x = self.conv3(x)
        return F.relu(out + x)
        

class ResNet(nn.HybridBlock):
    def __init__(self, num_classes, verbose=False, **kwargs):
        super(ResNet, self).__init__(**kwargs)
        self.verbose = verbose
        with self.name_scope():
            net = self.net = nn.HybridSequential()
            # block 1
            net.add(nn.Conv2D(channels=32, kernel_size=3, strides=1, padding=1))
            net.add(nn.BatchNorm())
            net.add(nn.Activation(activation='relu'))
            # block 2
            for _ in range(3):
                net.add(Residual(channels=32))
            # block 3
            net.add(Residual(channels=64, same_shape=False))
            for _ in range(2):
                net.add(Residual(channels=64))
            # block 4
            net.add(Residual(channels=128, same_shape=False))
            for _ in range(2):
                net.add(Residual(channels=128))
            # block 5
            net.add(nn.AvgPool2D(pool_size=18))
            net.add(nn.Flatten())
            net.add(nn.Dense(num_classes))
    
    def hybrid_forward(self, F, x):
        out = x
        for i, b in enumerate(self.net):
            out = b(out)
            if self.verbose:
                print ('Block %d output %s' % (i+1, out.shape))
        return out